<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">TFM - Previsión de demanda mediante uso de técnicas de machine learning
</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Carlos Pérez Cebrián</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

### SVR (Support Vector Regression) o SVM en regresión

Las Support Vector Machine (SVM) se fundamentan en el _Maximal Margin Classifier_, que a su vez, se basan en el concepto de hiperplano.

En un espacio p-dimensional, un hiperplano se define como un subespacio plano y afín de dimensiones p-1. El término afín significa que el subespacio no debe pasar por el origen. En un espacio de dos dimensiones, el hiperplano es un subespacio de 1 dimensión, es decir, una recta. En un espacio tridimensional, un hiperplano es un subespacio de dos dimensiones, un plano convencional. Para dimensiones p>3 no es intuitivo visualizar un hiperplano, pero el concepto de subespacio con p-1 dimensiones se mantiene.

La definición de hiperplano para casos perfectamente separables linealmente resulta en un número infinito de posibles hiperplanos, lo que hace necesario un método que permita seleccionar uno de ellos como clasificador óptimo.

La solución a este problema consiste en seleccionar como clasificador óptimo al que se conoce como _maximal margin hyperplane_ o hiperplano óptimo de separación, que se corresponde con el hiperplano que se encuentra más alejado de todas las observaciones de entrenamiento. Para obtenerlo, se debe calcular la distancia perpendicular de cada observación a un determinado hiperplano. La menor de estas distancias (conocida como margen) determina cómo de lejos está el hiperplano de las observaciones de entrenamiento. El _maximal margin hyperplane_ se define como el hiperplano que consigue un mayor margen, es decir, que la distancia mínima entre el hiperplano y las observaciones es lo más grande posible. Aunque esta idea suena razonable, no es posible aplicarla, ya que habría infinitos hiperplanos contra los que medir las distancias. En su lugar, se recurre a métodos de optimización.

El proceso de optimización tiene la peculiaridad de que sólo las observaciones que se encuentran justo al margen o que lo violan influyen sobre el hiperplano. A estas observaciones se les conoce como vectores soporte (_vectors suport_) y son las que definen el clasificador obtenido.

#### Los _kernels_ en SVM

Hay veces en que no hay manera de encontrar un hiperplano que permita separar dos clases. En estos casos decimos que las clases no son linealmente separables. Para resolver este problema podemos utilizar el truco del núcleo .

El truco del núcleo (_kernel trick_) consiste en utilizar una dimensión nueva en la que podamos encontrar un hiperplano para separar las clases. Se puede ver un un ejemplo en: https://www.youtube.com/watch?v=OdlNM96sHio

Al igual que en el algoritmo visto anteriormente (KNN), las SVM también dependen de varios hiperparámetros. 

En este caso intentaremos optimizar dos hiperparámetros:

  - **C**: es la regularización, es decir, el valor de penalización de los errores en la clasificación. Indica el compromiso entre obtener el hiperplano con el margen más grande posible y clasificar el máximo número de ejemplos correctamente. Probaremos los valores: 0.01, 0.1, 1, 10, 50, 100 y 200.
  
  - **Gama**: coeficiente que multiplica la distancia entre dos puntos en el kernel radial. Para decirlo a "grosso modo", cuanto más pequeño es gama, más influencia tienen dos puntos cercanos. Probaremos los valores: 0.001, 0.01, 0.1, 1 y 10.
  
Para validar el rendimiento del algoritmo con cada combinación de hiperparámetros utilizaremos validación cruzada (_cross-validation_) con 4 particiones estratificadas.

Cargamos las siguientes librerías necesarias:

In [7]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

Cargar el conjunto de datos:
(Nota: el conjunto de datos es el resultado de una fase de exploración, analisis y preparación previa.

In [8]:
# Cargar el archivo CSV con los datos de ventas
file_path = "Data/VentasRdo.csv"
file_path = "Data/VentasRdo_10productos.csv"
df = pd.read_csv(file_path, parse_dates=['idSecuencia'])

df['producto']    = LabelEncoder().fit_transform(df['producto'])
df['bolOpen']     = LabelEncoder().fit_transform(df['bolOpen'])
df['EnPromocion'] = LabelEncoder().fit_transform(df['EnPromocion'])
df['diasemana']   = LabelEncoder().fit_transform(df['diasemana'])
df['semana']      = LabelEncoder().fit_transform(df['semana'])

# Mostrar las primeras filas para verificar la carga
print(df.head())

   producto idSecuencia  udsVenta  bolOpen  EnPromocion  anyomes  diasemana  \
0         0  2022-12-06  0.000000        1            0        2          1   
1         1  2022-12-06  1.098612        1            0        2          1   
2         2  2022-12-06  0.000000        1            0        2          1   
3         3  2022-12-06  0.000000        1            0        2          1   
4         4  2022-12-06  0.000000        1            0        2          1   

   semana  media_7_dias  media_30_dias  venta_lag_1  venta_lag_7  venta_lag_30  
0      48      2.120606       2.301086     2.833213     2.079442           0.0  
1      48      1.874395       2.042219     2.079442     1.791759           0.0  
2      48      1.844732       1.681282     2.302585     2.564949           0.0  
3      48      1.186639       1.540582     1.098612     0.000000           0.0  
4      48      1.571729       1.871285     2.302585     2.564949           0.0  


Definir las características (X) y la variable objetivo (y)

In [9]:
# Definir las características (X) y la variable objetivo (y)
X = df[['producto', 'bolOpen', 'EnPromocion','anyomes', 'diasemana', 'semana','media_7_dias','media_30_dias','venta_lag_1','venta_lag_7','venta_lag_30']]  # Características
y = df['udsVenta']  # Variable objetivo


In [10]:
# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5608, 11)
(5608,)
(1402, 11)
(1402,)


El SVR y otros modelos basados en gradiente o distancias suelen funcionar mejor cuando las variables están normalizadas o estandarizadas. Escalar las variables permite que todas contribuyan de manera equivalente al cálculo de las distancias y evita que las de mayor magnitud dominen el proceso de optimización. 

Incluso si algunas variables están en la misma escala, aplicar el mismo escalado a todo el conjunto asegura una transformación uniforme y evita posibles inconsistencias.

In [11]:
# Escalado de datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
grid_params ={
    'C': [0.01, 0.1, 1, 10, 50, 100, 200],
    'gamma': [0.001, 0.01, 0.1, 1, 10]
}

gs_svm = GridSearchCV( SVR(),  grid_params, verbose=1, cv=4, n_jobs = -1,return_train_score = True)

In [13]:
gs_svm_results = gs_svm.fit(X_train, y_train)

Fitting 4 folds for each of 35 candidates, totalling 140 fits


In [14]:
print('best_score: ',gs_svm_results.best_score_)
print('best_estimador: ',gs_svm_results.best_estimator_)
print('best_paramas: ',gs_svm_results.best_params_)

resultados_svm = pd.DataFrame(gs_svm_results.cv_results_)
resultados_svm.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)

best_score:  0.4910101014800806
best_estimador:  SVR(C=50, gamma=0.01)
best_paramas:  {'C': 50, 'gamma': 0.01}


,param_C,param_gamma,mean_test_score,std_test_score,mean_train_score,std_train_score
21,50,0.01,0.491010,0.031884,0.527143,0.008925
26,100,0.01,0.489094,0.032636,0.533887,0.008938
16,10,0.01,0.489082,0.029669,0.510831,0.009888
11,1,0.01,0.489045,0.029367,0.498518,0.009237
7,0.1,0.1,0.487860,0.033298,0.516167,0.009307
25,100,0.001,0.487495,0.028940,0.496078,0.009792
30,200,0.001,0.486711,0.029227,0.497703,0.010287
20,50,0.001,0.486507,0.028503,0.493210,0.009391
31,200,0.01,0.485746,0.033653,0.540829,0.008984
15,10,0.001,0.477808,0.026543,0.481888,0.008568


In [16]:
#métricas
y_pred = gs_svm_results.best_estimator_.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R^2: {r2}")

RMSE: 0.711913186708125
MAE: 0.5294983604386788
R^2: 0.511896666005193
